# Haptic Classification Training

This notebook creates the Haptic Network Architecture and trains it for grasp testset1. After training the network weights will be stored in the folder `../Deep-Multi-Sensory-Object-Categorization_dataset/grasp_Freq_50TimePerSec_7channels_testset1_logs`

## Dependencies

`Python 3.5.4` is used for development and following packages are required to run the code provided in the notebook:

`pip install googledrivedownloader`<br>
`pip install matplotlib`<br>
`pip install tensorflow-gpu`<br>
`pip install numpy`

In [1]:
import pickle, os, csv, time, shutil
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from google_drive_downloader import GoogleDriveDownloader as gdd

In [2]:
def print_image(image, title):
    """Print the image

    :param image: image pixels in list
    :param title: title as string to be printed on top of the image
    """
    plt.imshow(image)
    plt.title(title)
    plt.colorbar()
    plt.show()

def time_taken(start, end):
    """Human readable time between `start` and `end`

    :param start: time.time()
    :param end: time.time()
    :returns: day:hour:minute:second
    """
    time = end-start
    day = time // (24 * 3600)
    time = time % (24 * 3600)
    hour = time // 3600
    time %= 3600
    minutes = time // 60
    time %= 60
    seconds = time
    day_hour_min_sec = str('%02d' % int(day))+":"+str('%02d' % int(hour))+":"+str('%02d' % int(minutes))+":"+str('%02d' % int(seconds))
    
    return day_hour_min_sec

In [3]:
"""
Downloading `category_object_labels.bin`.
This contains two dictionaries that stores category labels and object labels
"""

gdd.download_file_from_google_drive(file_id='15xPAbso4-uLk8PcRi9cb70r1r3FaU6ux',
                                    dest_path='../Deep-Multi-Sensory-Object-Categorization_dataset/category_object_labels.bin',
                                    unzip=False)

bin_file = open("../Deep-Multi-Sensory-Object-Categorization_dataset/category_object_labels.bin", "rb")

category_labels = pickle.load(bin_file)
object_labels = pickle.load(bin_file)

bin_file.close()

## Haptic Dataset

In [4]:
"""
Downloading testset1 for haptic of grasp interaction and reading it.
"""

db_file_name = "grasp_Freq_50TimePerSec_7channels_testset1.bin"
gdd.download_file_from_google_drive(file_id='16g4Cp67SHqpVSUyOryPEH2HCTZhMcCeB',
                                    dest_path='../Deep-Multi-Sensory-Object-Categorization_dataset/'+db_file_name,
                                    unzip=False)

bin_file = open("../Deep-Multi-Sensory-Object-Categorization_dataset/"+db_file_name, "rb")

haptic_frames_2d_train = pickle.load(bin_file)
category_label_train = pickle.load(bin_file)
object_label_train = pickle.load(bin_file)
category_label_train_one_hot = pickle.load(bin_file)
object_label_train_one_hot = pickle.load(bin_file)

haptic_frames_2d_test = pickle.load(bin_file)
category_label_test = pickle.load(bin_file)
object_label_test = pickle.load(bin_file)
category_label_test_one_hot = pickle.load(bin_file)
object_label_test_one_hot = pickle.load(bin_file)

bin_file.close()

In [5]:
"""
Printing shape of the haptic dataset
"""

batch_no = 188

print("batch, frames, channel: ", haptic_frames_2d_train.shape)

print("Category: ", list(category_labels.keys())[list(category_labels.values()).index(category_label_train[batch_no])])
print("Object: ", list(object_labels.keys())[list(object_labels.values()).index(object_label_train[batch_no])])

batch, frames, channel:  (400, 100, 7)
Category:  egg
Object:  egg_rough_styrofoam


## Building the Haptic Network Architecture

![Haptic Network Architecture](pics/Haptic_CNN.png)

## Haptic Network Hyper-parameters

To reduce over-fitting during training, we used dropout after the fully connected layer with 0.5 probability.
Training was performed with 400 training epochs using Adam optimization with a learning rate of 1 x $10^{-4}$.

In [6]:
# Network hyper-parameters
batch = 5
training_epochs = 400
display_step = 1

frames = time_step_size = haptic_frames_2d_train.shape[1]
channel = haptic_frames_2d_train.shape[2]

num_classes = category_label_train_one_hot.shape[1]

model_path = "../Deep-Multi-Sensory-Object-Categorization_dataset/"+db_file_name.split(".")[0]+"_logs/model.ckpt"
logs_path = "../Deep-Multi-Sensory-Object-Categorization_dataset/"+db_file_name.split(".")[0]+"_logs/"

X = tf.placeholder('float', [None, frames, channel], name='InputData')
Y = tf.placeholder('float', [None, num_classes], name='LabelData')
keep_prob = tf.placeholder_with_default(1.0, shape=(), name = 'keep')

In [7]:
skip_2nd_maxpool = ["grasp", "hold", "low"]

# Create model
def model():
    with tf.name_scope("Model"):
        data_placeholder = tf.placeholder('float', [None, frames, channel], name='InputData')
        
        net = tf.reshape(data_placeholder, [-1, frames, channel, 1])
        net = tf.layers.conv2d(inputs=net, filters=32, kernel_size=[20, 5], padding="same", activation=tf.nn.relu)
        net = tf.layers.max_pooling2d(inputs=net, pool_size=[10, 1], strides=2)
        net = tf.layers.conv2d(inputs=net, filters=64, kernel_size=[1, 3], padding="same", activation=tf.nn.relu)
        if (db_file_name.split("_")[0]) in skip_2nd_maxpool:
            net = tf.layers.max_pooling2d(inputs=net, pool_size=[1, 1], strides=[1, 2])
        else:
            net = tf.layers.max_pooling2d(inputs=net, pool_size=[10, 1], strides=2)
        net = tf.layers.flatten(net)
        # Dense Layer
        net = tf.layers.dense(inputs=net, units=1024, activation=tf.nn.relu)
        # Add dropout operation
        net = tf.layers.dropout(inputs=net, rate=keep_prob)
        logits = tf.layers.dense(inputs=net, units=num_classes)
        
    return logits

def loss(prediction, label_placeholder):
    with tf.name_scope('Loss'):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=label_placeholder))
        # Create a summary to monitor cost tensor
        cost_scalar = tf.summary.scalar("loss", cost)
    return cost, cost_scalar

def training(prediction, label_placeholder):
    with tf.name_scope('Optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_op = optimizer.minimize(cost)
    return train_op

def evaluate(prediction, Y):
    with tf.name_scope('Accuracy'):
        # Test model
        correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
        # Create a summary to monitor accuracy tensor
        accuracy_scalar = tf.summary.scalar("accuracy", accuracy)
    return accuracy, accuracy_scalar

In [8]:
"""
Creating the Neural Network
"""

model_dict = {}

prediction = model()
model_dict["Model"] = prediction

cost, cost_scalar = loss(prediction, Y)
model_dict["Loss"] = cost
model_dict["Loss_scalar"] = cost_scalar

train_op = training(prediction, Y)
model_dict["Optimizer"] = train_op

eval_op, accuracy_scalar = evaluate(prediction, Y)
model_dict["Accuracy"] = eval_op
model_dict["Accuracy_scalar"] = accuracy_scalar

print("model_dict: ", model_dict)

# Initializing the variables
init = tf.global_variables_initializer()

# 'Saver' op to save and restore all the variables
saver = tf.train.Saver(max_to_keep=1)

model_dict:  {'Model': <tf.Tensor 'Model/dense_1/BiasAdd:0' shape=(?, 20) dtype=float32>, 'Accuracy_scalar': <tf.Tensor 'Accuracy/accuracy:0' shape=() dtype=string>, 'Accuracy': <tf.Tensor 'Accuracy/Mean:0' shape=() dtype=float32>, 'Loss': <tf.Tensor 'Loss/Mean:0' shape=() dtype=float32>, 'Loss_scalar': <tf.Tensor 'Loss/loss:0' shape=() dtype=string>, 'Optimizer': <tf.Operation 'Optimizer/Adam' type=NoOp>}


In [9]:
if os.path.exists(logs_path):
    shutil.rmtree(logs_path)
    os.makedirs(logs_path)
else:
    os.makedirs(logs_path)

In [10]:
"""
Writing 'Time', 'Epoch', 'Cost', 'Accuracy' in CSV file
"""

epoch_cost_accuracy = []
epoch_cost_accuracy.append("Time")
epoch_cost_accuracy.append("Epoch")
epoch_cost_accuracy.append("Cost")
epoch_cost_accuracy.append("Accuracy")

with open(logs_path+db_file_name.split(".")[0]+"_data.csv",'w') as f:
    writer = csv.writer(f, lineterminator="\n")
    writer.writerow(epoch_cost_accuracy)

## Training

In [11]:
# Start Training

start_time = time.time()

with tf.Session() as sess:
    # Initialize variables
    sess.run(init)
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost_list = 0.0
        total_batch = int(len(haptic_frames_2d_train)/batch)
        
        # Shuffle data
        shuffle_indices = np.arange(len(haptic_frames_2d_train))
        np.random.shuffle(shuffle_indices)
        
        i = 0
        # Loop over all batches
        for start, end in zip(range(0, len(haptic_frames_2d_train), batch), range(batch, len(haptic_frames_2d_train)+1, batch)):
            input_data, label_data = haptic_frames_2d_train[shuffle_indices][start:end], category_label_train_one_hot[shuffle_indices][start:end]
            X = tf.get_default_graph().get_tensor_by_name("Model/InputData:0")
            _, new_cost, cost_scalar = sess.run([model_dict["Optimizer"], model_dict["Loss"], model_dict["Loss_scalar"]], feed_dict={X: input_data, Y: label_data, keep_prob: 0.5})
            # Compute average loss
            avg_cost_list += new_cost/total_batch
            summary_writer.add_summary(cost_scalar, epoch * total_batch + i)

            i += 1
            
        save_path = saver.save(sess, model_path, epoch)
         
        # Calculate Accuracy
        avg_accuracy_list = 0.0
        total_batch = int(len(haptic_frames_2d_test)/batch)
        i = 0
        for start, end in zip(range(0, len(haptic_frames_2d_test), batch), range(batch, len(haptic_frames_2d_test)+1, batch)):
            input_data, label_data = haptic_frames_2d_test[start:end], category_label_test_one_hot[start:end]
            X = tf.get_default_graph().get_tensor_by_name("Model/InputData:0")
            accuracy, accuracy_scalar = sess.run([model_dict["Accuracy"], model_dict["Accuracy_scalar"]], feed_dict={X: input_data, Y: label_data, keep_prob: 1.0})
            # Compute average accuracy
            avg_accuracy_list += accuracy/total_batch
            summary_writer.add_summary(accuracy_scalar, epoch * total_batch + i)
            
            i += 1
        
        # Printing current epoch accuracy
        epoch_cost_accuracy = []
        epoch_cost_accuracy.append(time_taken(start_time, time.time()))
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), ", Time: ", time_taken(start_time, time.time()))
            a_string = "Cost - "
            epoch_cost_accuracy.append(epoch+1)
            
            a_string += str(avg_cost_list)
            epoch_cost_accuracy.append(str(avg_cost_list))
            
            a_string = a_string[0:-2]+" --> Accuracy - "
            a_string += str(avg_accuracy_list)
            epoch_cost_accuracy.append(str(avg_accuracy_list))
            
            print(a_string)
        
        # Writing current epoch data
        with open(logs_path+db_file_name.split(".")[0]+"_data.csv", 'a') as f: # append to the file created
            writer = csv.writer(f, lineterminator="\n")
            writer.writerow(epoch_cost_accuracy)
    
    print("Optimization Finished!")
    end_time = time.time()
    print("Time taken: day, hour, minutes, seconds->", time_taken(start_time, end_time))

Epoch: 0001 , Time:  00:00:00:05
Cost - 2.9826790153980 --> Accuracy - 0.11000000089406968
Epoch: 0002 , Time:  00:00:00:07
Cost - 2.74463790655135 --> Accuracy - 0.20000000447034835
Epoch: 0003 , Time:  00:00:00:09
Cost - 2.550800698995 --> Accuracy - 0.22000000625848773
Epoch: 0004 , Time:  00:00:00:10
Cost - 2.4096208214759 --> Accuracy - 0.2300000071525574
Epoch: 0005 , Time:  00:00:00:12
Cost - 2.3001615822315 --> Accuracy - 0.2900000050663949
Epoch: 0006 , Time:  00:00:00:14
Cost - 2.2080246150493 --> Accuracy - 0.2300000049173832
Epoch: 0007 , Time:  00:00:00:16
Cost - 2.13350084275007 --> Accuracy - 0.30000000819563866
Epoch: 0008 , Time:  00:00:00:19
Cost - 2.05816850066184 --> Accuracy - 0.2900000058114529
Epoch: 0009 , Time:  00:00:00:21
Cost - 1.96316113024950 --> Accuracy - 0.32000000551342966
Epoch: 0010 , Time:  00:00:00:22
Cost - 1.88551145121455 --> Accuracy - 0.33000000491738324
Epoch: 0011 , Time:  00:00:00:25
Cost - 1.84001259654760 --> Accuracy - 0.3500000052154065

Epoch: 0091 , Time:  00:00:03:15
Cost - 0.236474444624036 --> Accuracy - 0.34000000730156904
Epoch: 0092 , Time:  00:00:03:17
Cost - 0.203164425626164 --> Accuracy - 0.36000000908970836
Epoch: 0093 , Time:  00:00:03:19
Cost - 0.193653672875370 --> Accuracy - 0.3800000086426735
Epoch: 0094 , Time:  00:00:03:21
Cost - 0.29980148155445 --> Accuracy - 0.3500000089406967
Epoch: 0095 , Time:  00:00:03:23
Cost - 0.176312372006941 --> Accuracy - 0.3500000089406967
Epoch: 0096 , Time:  00:00:03:25
Cost - 0.174683784920489 --> Accuracy - 0.3300000078976154
Epoch: 0097 , Time:  00:00:03:27
Cost - 0.193231571267824 --> Accuracy - 0.37000000923871995
Epoch: 0098 , Time:  00:00:03:29
Cost - 0.159030793106649 --> Accuracy - 0.4000000111758709
Epoch: 0099 , Time:  00:00:03:32
Cost - 0.150200986384879 --> Accuracy - 0.4100000075995922
Epoch: 0100 , Time:  00:00:03:34
Cost - 0.174219781591091 --> Accuracy - 0.39000000804662704
Epoch: 0101 , Time:  00:00:03:36
Cost - 0.168640668841544 --> Accuracy - 0.41

Epoch: 0180 , Time:  00:00:06:19
Cost - 0.0175091790588339 --> Accuracy - 0.42000000923871994
Epoch: 0181 , Time:  00:00:06:21
Cost - 0.05126312818974 --> Accuracy - 0.42000000998377807
Epoch: 0182 , Time:  00:00:06:23
Cost - 0.303196509715053 --> Accuracy - 0.37000000923872
Epoch: 0183 , Time:  00:00:06:25
Cost - 0.22663727577019 --> Accuracy - 0.3800000071525574
Epoch: 0184 , Time:  00:00:06:27
Cost - 0.124711087156902 --> Accuracy - 0.3700000084936619
Epoch: 0185 , Time:  00:00:06:29
Cost - 0.056892435862027 --> Accuracy - 0.3800000078976154
Epoch: 0186 , Time:  00:00:06:31
Cost - 0.0254481730138650 --> Accuracy - 0.3900000095367432
Epoch: 0187 , Time:  00:00:06:33
Cost - 0.0219020197197096 --> Accuracy - 0.38000000789761545
Epoch: 0188 , Time:  00:00:06:35
Cost - 0.011913199931586 --> Accuracy - 0.3800000101327896
Epoch: 0189 , Time:  00:00:06:37
Cost - 0.009302235026552 --> Accuracy - 0.38000000938773154
Epoch: 0190 , Time:  00:00:06:39
Cost - 0.0089363906721700 --> Accuracy - 0.3

Epoch: 0268 , Time:  00:00:09:17
Cost - 0.063556833802431 --> Accuracy - 0.4100000098347665
Epoch: 0269 , Time:  00:00:09:19
Cost - 0.0255325161466316 --> Accuracy - 0.41000001132488256
Epoch: 0270 , Time:  00:00:09:21
Cost - 0.0092764926008385 --> Accuracy - 0.41000001132488256
Epoch: 0271 , Time:  00:00:09:23
Cost - 0.0066378803108818 --> Accuracy - 0.42000001147389415
Epoch: 0272 , Time:  00:00:09:25
Cost - 0.00581928121209785 --> Accuracy - 0.4100000105798245
Epoch: 0273 , Time:  00:00:09:27
Cost - 0.0051438823422358 --> Accuracy - 0.4100000105798245
Epoch: 0274 , Time:  00:00:09:29
Cost - 0.0046535458433936 --> Accuracy - 0.4100000105798245
Epoch: 0275 , Time:  00:00:09:31
Cost - 0.0042913617293379 --> Accuracy - 0.40000001043081285
Epoch: 0276 , Time:  00:00:09:33
Cost - 0.0040754096286036 --> Accuracy - 0.40000001043081285
Epoch: 0277 , Time:  00:00:09:35
Cost - 0.0037238882061501 --> Accuracy - 0.40000001043081285
Epoch: 0278 , Time:  00:00:09:37
Cost - 0.00358889652634388 --> 

Epoch: 0356 , Time:  00:00:12:15
Cost - 0.00103639511553410 --> Accuracy - 0.3900000095367432
Epoch: 0357 , Time:  00:00:12:17
Cost - 0.00099812230600946 --> Accuracy - 0.3900000095367432
Epoch: 0358 , Time:  00:00:12:20
Cost - 0.00095492221817039 --> Accuracy - 0.3900000095367432
Epoch: 0359 , Time:  00:00:12:22
Cost - 0.00092248027767709 --> Accuracy - 0.3900000095367432
Epoch: 0360 , Time:  00:00:12:23
Cost - 0.0008994639621960 --> Accuracy - 0.3900000095367432
Epoch: 0361 , Time:  00:00:12:25
Cost - 0.00093416981744667 --> Accuracy - 0.3900000095367432
Epoch: 0362 , Time:  00:00:12:27
Cost - 0.00085579796141246 --> Accuracy - 0.3900000095367432
Epoch: 0363 , Time:  00:00:12:30
Cost - 0.00083786011146003 --> Accuracy - 0.3900000095367432
Epoch: 0364 , Time:  00:00:12:32
Cost - 0.0007959679909617 --> Accuracy - 0.3900000095367432
Epoch: 0365 , Time:  00:00:12:34
Cost - 0.00079673861582705 --> Accuracy - 0.3900000095367432
Epoch: 0366 , Time:  00:00:12:35
Cost - 0.00073666565158418 --